In [10]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

file_name = "actors_jpbox"
version="1"

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="72eb7803-e874-44cb-b6d9-33f2fa3eb88c",
    resource_group_name="vpoutotrg",
    workspace_name="mlstudio-groupe4"
)

data_asset = ml_client.data.get(file_name, version=version)

df = pd.read_parquet(data_asset.path)

df.head()

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,amount_of_film_played_in,boxoffice_average,boxoffice_average_first_role,boxoffice_total,boxoffice_total_first_role,name,nationality,rank
0,15,197655,284469,3360134,2275749,Jason Segel,américain,551
1,18,99462,94796,3083328,1801124,Kate Hudson,américaine,552
2,16,81984,103371,3033394,1964043,Mira Sorvino,américaine,553
3,11,220324,211238,2643888,1056192,Alison Lohman,américaine,554
4,9,92895,121477,1579207,1579207,Tim Allen,américain,555


In [11]:
df["nationality"] = df["nationality"].apply(lambda x : x.lower())

In [12]:
df["nationality"].value_counts()

nationality
américain        176
américaine        93
français          82
française         54
britannique       45
belge              9
irlandais          7
australien         5
australienne       4
espagnol           3
chinois            3
espagnole          2
canadien           2
chilien            1
mexicain           1
mexicaine          1
suédoise           1
italien            1
israélienne        1
néo-zélandais      1
autrichien         1
allemande          1
danois             1
sud-africaine      1
belgique           1
italienne          1
autrichienne       1
Name: count, dtype: int64

In [13]:
nationality_genres = [
    ("américain", "américaine"),
    ("français", "française"),
    ("australien", "australienne"),
    ("espagnol", "espagnole"),
    ("belgique", "belge"),
    ("mexicain", "mexicaine"),
    ("autrichien", "autrichienne"),
    ("italien", "italienne")
]

def fuck_patriarchy(x):
    for item in nationality_genres:
        if x.lower() == item[0]:
            return item[1]
    return x

df["nationality"] = df["nationality"].apply(fuck_patriarchy)

In [14]:
df["nationality"].value_counts()

nationality
américaine       269
française        136
britannique       45
belge             10
australienne       9
irlandais          7
espagnole          5
chinois            3
mexicaine          2
italienne          2
canadien           2
autrichienne       2
chilien            1
suédoise           1
israélienne        1
néo-zélandais      1
allemande          1
danois             1
sud-africaine      1
Name: count, dtype: int64

In [15]:
df.head()

,amount_of_film_played_in,boxoffice_average,boxoffice_average_first_role,boxoffice_total,boxoffice_total_first_role,name,nationality,rank
0,15,197655,284469,3360134,2275749,Jason Segel,américaine,551
1,18,99462,94796,3083328,1801124,Kate Hudson,américaine,552
2,16,81984,103371,3033394,1964043,Mira Sorvino,américaine,553
3,11,220324,211238,2643888,1056192,Alison Lohman,américaine,554
4,9,92895,121477,1579207,1579207,Tim Allen,américaine,555


In [16]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import pyarrow as pa
import pyarrow.parquet as pq

df_to_blob = df
path = 'allocine_spider_clean.parquet'
file_name= "actors_jpbox" # Le même nom qu'à la step d'avant
version = "2.0 " # Ici je me dit qu'on pourait se mettre d'accord sur lesquels sont les notres, genres 2.truc c'est vic, 3.truc ludi, 4.truc raouf ?
description = "premier jet"

table = pa.Table.from_pandas(df_to_blob)
pq.write_table(table, path)
new_data_asset = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    name=file_name, 
    version=version,           
    description=description
)

ml_client.data.create_or_update(new_data_asset)

HttpResponseError: (UserError) A data version with this name and version already exists. If you are trying to create a new data version, use a different name or version. If you are trying to update an existing data version, the existing asset's data uri cannot be changed. Only tags, description, and isArchived can be updated.
Code: UserError
Message: A data version with this name and version already exists. If you are trying to create a new data version, use a different name or version. If you are trying to update an existing data version, the existing asset's data uri cannot be changed. Only tags, description, and isArchived can be updated.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "672f7c8439badb38a1f8bf4ab59076df",
        "request": "a4c9533fcb2d694b"
    }
}Type: Environment
Info: {
    "value": "francecentral"
}Type: Location
Info: {
    "value": "francecentral"
}Type: Time
I

In [17]:
df_to_blob

,amount_of_film_played_in,boxoffice_average,boxoffice_average_first_role,boxoffice_total,boxoffice_total_first_role,name,nationality,rank
0,15,197655,284469,3360134,2275749,Jason Segel,américaine,551
1,18,99462,94796,3083328,1801124,Kate Hudson,américaine,552
2,16,81984,103371,3033394,1964043,Mira Sorvino,américaine,553
3,11,220324,211238,2643888,1056192,Alison Lohman,américaine,554
4,9,92895,121477,1579207,1579207,Tim Allen,américaine,555
...,...,...,...,...,...,...,...,...
494,45,1597288,1787895,76669820,75091569,Tom Cruise,américaine,40
495,55,1335686,1109952,76134103,48837888,Marion Cotillard,française,41
496,42,1630937,2274049,73392185,38658825,Omar Sy,française,42
497,48,1243655,364560,73375624,5832957,Gary Oldman,britannique,43


In [18]:
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="72eb7803-e874-44cb-b6d9-33f2fa3eb88c",
    resource_group_name="vpoutotrg",
    workspace_name="mlstudio-groupe4"
)

data_asset = ml_client.data.get("allo_cine", version="2.2.1")

df = pd.read_parquet(data_asset.path)
df = df[df['french_first_week_boxoffice'].notna()]
df["date"] = pd.to_datetime(df["date"], format='%Y-%m-%d', errors='coerce')
df = df[df['date'].notna()]
df['us_first_week_boxoffice'] = df['us_first_week_boxoffice'].fillna(0)
df.head()

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",4.4,2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,0.0,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",3.5,2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,0.0,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html
2,[Thom Hoffman],2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,0.0,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",3.3,2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,0.0,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...


In [19]:
df.dtypes

actors                                 object
critics_score                         float64
date                           datetime64[ns]
directors                              object
editor                                 object
genre                                  object
langage                                object
length                                float64
nationality                            object
viewers_score                         float64
us_first_week_boxoffice               float64
french_first_week_boxoffice           float64
french_visa                            object
title                                  object
vo_title                               object
url                                    object
dtype: object

In [20]:
df2 = df[df["actors"].isna()]

In [21]:
for item in df["actors"]:
    print(type(item))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [22]:
df_machin = df['actors'].dropna()

In [ ]:
import numpy as np

big_set_actors = set(["Thom Hoffman"])

for item in df_machin:
    for truc in item:
        big_set_actors.add(truc.replace(" ", "").replace("-", "").replace("_", "").strip().lower()) 

print(big_set_actors)


{'yoshihiko hakamada', 'maurice roëves', 'tony plana', 'greg grunberg', 'guido mannari', 'tom mcgrath', 'liu min', 'angel oquendo', 'uwe steinmetz', 'b.d. wong', 'patrick gibson', 'julia nickson', 'rob steiner', 'chris diamantopoulos', 'anne meara', 'timothy hutton', 'dimitri rougeul', 'marinko prga', 'michela cescon', 'carl gilliard', 'billie whitelaw', 'lee oakes', 'richard lukunku', 'kelly washington', 'mariana paola vicente', 'eric toledano', 'audrey tautou', 'paul hansen kim', 'claire danes', 'ana esther alborg', 'paul vogt', 'jayam ravi', 'françoise remont', 'yuming hey', 'connie ray', 'bernard marbaix', 'harold perrineau', 'thomas m. wright', 'andy buckley', 'emmanuel curtil', 'kana kita', 'shiri appleby', 'steve rodgers', 'john spencer', 'luc leclerc du sablon', 'tony vogel', 'ian hallard', 'ben attal', 'hector luis bustamante', 'veniamin kats', 'veena sood', 'steven mackintosh', 'eden malyn', 'maria jurado', 'gustavo hernandez', 'vera chok', 'péter takátsy', 'benoît giros', 'm

In [29]:
yapas = []

for item in df_to_blob['name']:
    if item.lower() in big_set_actors:
        pass
    else :
        yapas.append(item)

In [25]:
yapas

['Gael Garcia Bernal',
 'Linda Fiorentino',
 'Fabrice Eboue',
 'Lorant Deutsch',
 'Renee Zellweger',
 'Yun-Fat Chow',
 'Sergi Lopez',
 'Ellen Page',
 'Sean William Scott',
 'Jean-Baptiste Lucien',
 'Roberto Begnini',
 'Marilyn Monroe',
 'Jérémy Renier',
 'Christophe Lambert',
 'Michael Youn',
 'Elodie Bouchez',
 'Pascal Legitimus',
 'Clark Gable',
 'Humphrey Bogart',
 '- Coluche',
 'Samy Nacéri',
 'Oliver Gourmet',
 'Patrick Dewaere',
 'Penelope Cruz',
 'Angela Basset',
 'Robert Downey Jr',
 'Robert Mitchum',
 'Simone Signoret',
 'Steve McQueen',
 'John Wayne',
 'Charles Bronson',
 'Romy Schneider']

In [26]:
yapas = [x.strip() for x in yapas]

In [30]:
len(yapas)

39